In [ ]:
# import numpy as np
# import pandas as pd

# import statsmodels.api as sm
# import statsmodels.formula.api as smf

# from scipy import stats
# from patsy import dmatrices
# from statsmodels.stats.stattools import durbin_watson

# import matplotlib.pyplot as plt

### Detekcja obserwacji odstających:

1. **Wykres studentyzowanych rezyduów**

Dla małych prób, wartości zmiennej objaśniającej nie są w miarę równomiernie rozłożone i niektóre błędy $\text{SE}(e_i)$ mogą znacznie odbierać od błędu $S$. Wówczas dobrze jest analizować rezydua przy użyciu tzw. **rezyduów studentyzowanych**.

$$r_i =\frac{e_i}{\text{SE}(e_i)}$$

To pozwoli wykrywać obserwacje faktycznie odstające, pomijając te, które przy analizie rezyduów $e_i$ sugerowały, że są odstające mimo, że takimi nie były. Dla rezyduów studentyzowanych zakłada się, że przy poziomie ufności równym 0.95 uznaje się je za normalne (zachowujące własność rozkładu normalnego), gdy należą do przedziału $[−2,+2]$.

Wykres studentyzowanych rezyduów względem ich indeksu identyfikuje duże wartości, które przypuszczalnie odpowiadają obserwacjom odstającym. Metodata nie sprawdzi się w sytuacji, gdy mamy w analizowanym zbiorze obserwację wpływową o małej wartości $e_i$. Wówczas bowiem nie określimy jej jako odstającej mimo, że taka w istocie jest.

2. **Wpływowość**

Wpływ $i$-tej obserwacji $h_i$ określamy wzorem
$$
h_i = \frac{1}{n} + \frac{(x_i - \overline{x})^2}{\sum_{i=1}^n(x_i - \overline{x})^2},
$$
który określa odstępstwo $x_i$ od $\overline{x}$.

Dla modelu o $p$ parametrach (gdzie $p$ to łączna liczba zmiennych objaśniających i objaśnianych), obserwację uznajemy za wpływową jeśli
$$
h_i \geq \frac{2p}{n}.
$$

3. **Odległość Cooka**

Jest to miara stopnia zmiany współczynników regresji, gdyby dany przypadek pominąć w obliczeniach współczynników:
$$
D_i = \frac{\sum_{j=1}^n(\hat{Y}_j - \hat{Y}_{j(i)})^2}{pS^2},
$$
gdzie $\hat{Y}_j$ - prognoza na podstawie pełnych danych, $\hat{Y}_{j(i)}$ - prognoza bez $i$-tej obserwacji.

**Interpretacja**: Duża wartość $D_i$ wskazuje na znaczy wpływ usunięcia $i$-tej obserwacji, czyli $i$-ta obserwacja jest obserwacją wpływową.

Wszystkie wartości dla danej odległości powinny być tego samego rzędu. Jeśli tak nie jest, to prawdopodobnie dany przypadek ma istotnie duży wpływ na obciążenie równania regresji.

**Reguła kciuka**: $D_i > \frac{4}{(n − p − 1)}$

# Zadanie 1

Wczytaj zbiór `Carseats`, który zawierają dane o sprzedaży fotelików samochodowych pewnej firmy w 400 różnych lokalizacjach:

• sprzedaż jednostek (w tysiącach) w danej lokalizacji (`Sales`),

• cena sprzedaży konkurenta w danej lokalizacji (`CompPrice`),

• poziom dochodów społeczności wdanej lokalizacji w tys. dolarów (`Income`),

• wielkość populacji w danym regionie (`Population`),

• lokalny budżet reklamowy dla firmy w danej lokalizacji (`Advertising`),

• cena sprzedaży w danej lokalizacji (`Price`),

• na ile korzystne jest miejsce na półkach sklepu dla fotelików w danym regionie?
(`Shelveloc`),

• średni wiek ludności lokalnej (`Age`),

• poziom wykształcenia w każdej lokalizacji (`Education`),

• czy położenie sklepu jest w lokalizacji miejskiej? (`Urban`)

• czy sklep znajduje się w USA? (`US`)

1. Wyznacz model liniowy najlepiej opisujący zależność sprzedaży fotelików samochodowych od pozostałych dostępnych zmiennych.

2. Zidentyfikuj obserwacje odstające w modelu.

3. Zidentyfikuj obserwacje wpływowe.

In [ ]:
# carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)
# columns = list(carseats.data.columns)
# columns.remove('Sales')
# features = "+".join(columns)

# model = smf.ols('Sales ~ ' + features, data=carseats.data)
# fitted_model = model.fit()
# fitted_model.summary()

In [ ]:
# h = fitted_model.get_influence()
# h.resid_studentized

In [ ]:
# #wartości odstające
# x = np.arange(1, h.resid_studentized.size + 1)
# plt.scatter(x=x, y=h.resid_studentized)
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=-2, color="r")
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=0, color="r")
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=2, color="r")
# #te poza pasem kwalifikujemy jako do usunięcia

In [ ]:
# #wartości wpływowe h_i
# h.hat_matrix_diag
# threshold = 2*h.exog.shape[1]/h.exog.shape[0]

# x = np.arange(1, h.hat_matrix_diag.size + 1)
# plt.scatter(x=x, y=h.hat_matrix_diag)
# plt.hlines(xmin=1, xmax=h.hat_matrix_diag.size + 1, y=threshold, color="r")
# #są dwie obs wpływowe

In [ ]:
# #obserwacje wpływowe odległości cooka (inna metoda detekcji wpływowości)
# threshold = 4/(h.exog.shape[0]-h.exog.shape[1]-1)
# h.cooks_distance
# x = np.arange(1, h.cooks_distance[0].size + 1)
# plt.scatter(x=x, y=h.cooks_distance[0])
# plt.hlines(xmin=1, xmax = x[-1], y = threshold, color='r')

# Zadanie 2
Wczytaj dane `pollution.csv` (opis danych: https://www.rdocumentation.org/packages/SMPracticals/versions/1.4-3/topics/pollution). Dopasuj model regresji liniowej postaci: `mort~nox`.
1. Czy model jest dobrze dopasowany do danych? Jeżeli nie, dlaczego?
2. Dokonaj logarytmicznego przekształcenia zmiennej objaśniającej. Jak mocno poprawia się współczynnik $R^2$?
3. Dla przekształconego modelu usuń obserwacje odstające. Jak mocno poprawia się współczynnik $R^2$?

In [ ]:
# pollution = pd.read_csv("pollution.csv")
# x = pollution['nox']
# x1 = np.array(x).reshape(-1,1)
# y = pollution['mort']
# lm1=LinearRegression()
# lm1.fit(x1,y)
# lm1.score(x1,y)

# Zadanie 3

1. Wczytać dane `realest.csv`, które zawierają informacje na temat domów na przedmieściach Chicago: cena domu (`Price`), liczba sypialni (`Bedroom`), powierzchnia w stopach kwadratowych (`Space`), liczba pokoi (`Room`), szerokość frontu działki w stopach (`Lot`), roczny podatek od nieruchomości (`Tax`), liczba łazienek (`Bathroom`), liczba miejsc parkingowych w garażu (`Garage`) i stan domu (`Condition`, 0-dobry, 1-wymaga remontu).
       
2. Wyznacz model liniowy najlepiej opisujący zależność ceny domu od pozostałych zmiennych w zbiorze. Oblicz $R^2$ tego modelu.

3. Znajdź obserwacje wpływowe w modelu i usuń je. Dopasuj model bez tych obserwacji. Oblicz $R^2$ tego modelu.

4. Znajdź obserwacje odstające w nowym modelu i usuń je. Dopasuj nowy model. Oblicz $R^2$ tego modelu.

In [ ]:
# realest = pd.read_csv('realest.csv')
# #są Na wyrzućmy je
# realest = realest.dropna(axis =0)
# print(realest)